In [1]:
#The task is to use Word embeddings in Spacy
#Spacy is a natural language processing library for Python designed to have fast performance
#Spacy is a free, open-source library for NLP in Python 
#and provides pre-trained word vectors(set of numbers) or word embedding models built in. 
#Downloaded a pre-defined model 
#python -m spacy download en_core_web_lg
#It’s a quick and easy technique for learning numberical representation of words
#word2vec - it is a technique which allows you to do mathematics with a word 
#(basically assign vector of numbers to a word)

#Gensim is a topic modelling library for Python that provides modules for training Word2Vec and other word embedding algorithms, and allows using pre-trained models.
import os
from os import listdir
import time
import gzip
import json 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics

import spacy
# Torch libs
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from torch.utils.data.sampler import WeightedRandomSampler

import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torchvision.io import read_image
import torch.optim as optim

from sklearn.model_selection import train_test_split

# Data libs
import pandas as pd
import numpy as np
# import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

In [2]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


In [9]:
os.chdir('data')

In [10]:
df = getDF('Luxury_Beauty_5.json.gz')

In [11]:
#No. of reviews/counts per rating
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34278 entries, 0 to 34277
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         34278 non-null  float64
 1   verified        34278 non-null  bool   
 2   reviewTime      34278 non-null  object 
 3   reviewerID      34278 non-null  object 
 4   asin            34278 non-null  object 
 5   style           16841 non-null  object 
 6   reviewerName    34278 non-null  object 
 7   reviewText      34265 non-null  object 
 8   summary         34263 non-null  object 
 9   unixReviewTime  34278 non-null  int64  
 10  vote            6532 non-null   object 
 11  image           617 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 3.2+ MB


In [12]:
display(df.head(10))

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"01 5, 2018",A2HOI48JK8838M,B00004U9V2,{'Size:': ' 0.9 oz.'},DB,This handcream has a beautiful fragrance. It d...,Beautiful Fragrance,1515110400,NaN,NaN
1,5.0,True,"04 5, 2017",A1YIPEY7HX73S7,B00004U9V2,{'Size:': ' 3.5 oz.'},Ajaey,"wonderful hand lotion, for seriously dry skin,...",wonderful hand lotion,1491350400,NaN,NaN
2,5.0,True,"03 27, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 250 g'},D. Jones,"Best hand cream around. Silky, thick, soaks i...",Best hand cream around,1490572800,NaN,NaN
3,5.0,True,"03 20, 2017",A2R4UNHFJBA6PY,B00004U9V2,{'Size:': ' 3.5 oz.'},Amazon Customer,Thanks!!,Five Stars,1489968000,NaN,NaN
4,5.0,True,"02 28, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 0.9 oz.'},D. Jones,Great hand lotion. Soaks right in and leaves ...,Great hand lotion!,1488240000,NaN,NaN
5,5.0,True,"02 25, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 250 g'},Amr,Great product. Doesn't leave you hands feeling...,Five Stars,1487980800,NaN,NaN
6,5.0,True,"02 25, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 3.5 oz.'},Amr,Great product. Doesn't leave you hands feeling...,Five Stars,1487980800,NaN,NaN
7,5.0,True,"01 30, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 0.9 oz.'},Amr,Just as described. Arrived on time.,Five Stars,1485734400,NaN,NaN
8,4.0,False,"01 24, 2017",A1YY53NQXFKMRN,B00004U9V2,{'Size:': ' 3.5 oz.'},Trixie,Nice lightweight hand cream for the summer.,"Smells good, absorbs quickly",1485216000,NaN,NaN
9,5.0,True,"12 1, 2016",A3R0NQ9E53JHYQ,B00004U9V2,{'Size:': ' 250 g'},T. Hooth,Best hand cream ever.,Five Stars,1480550400,NaN,NaN


In [13]:
display(df.tail(10))

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
34268,5.0,True,"08 21, 2018",A1ID5QT86H8J9D,B01GOZ61O8,NaN,JSM,Love it. Just wish it came it bigger tube. I...,Great SPF!,1534809600,NaN,NaN
34269,4.0,False,"05 21, 2018",A1G60ANYI0HMEU,B01GOZ61O8,NaN,Jeddy 3,"I really like the Tizo sunscreens, but this on...","Not sure what else it's doing, but it's defini...",1526860800,NaN,NaN
34270,3.0,False,"12 10, 2017",ATN4J5VS8Q0YM,B01GOZ61O8,NaN,Anna Hope,I used this on a recent trip across the countr...,Not Everyday,1512864000,NaN,NaN
34271,4.0,False,"10 10, 2017",AVL5IDEE8YA3D,B01GOZ61O8,NaN,Rachel McElhany,This product doesn't really say what it is. Th...,TIZO AM Replenish,1507593600,NaN,NaN
34272,2.0,False,"09 9, 2017",A2X2WTEVCZ5L8N,B01GOZ61O8,NaN,Sandy Kay,I'm not sure whether it is the ceramicides or ...,Hated how it felt on my face,1504915200,NaN,NaN
34273,4.0,False,"09 3, 2017",A2CF66KIQ3RKX3,B01GOZ61O8,NaN,Vivian Deliz,I like to use moisturizers and sunscreens that...,Works great as a moisturizer and sunscreen,1504396800,NaN,NaN
34274,4.0,False,"09 3, 2017",A1LKOIZXPQ9VG0,B01GOZ61O8,NaN,Elisa 20,I wouldn't be able to afford this if not asked...,Nice skin care product and sunscreen if you do...,1504396800,NaN,NaN
34275,1.0,True,"08 25, 2017",AV2RWORXTFRJU,B01H353HUY,NaN,Gapeachmama,Did nothing,One Star,1503619200,NaN,NaN
34276,5.0,False,"07 8, 2017",A22S7D0LP8GRDH,B01H353HUY,NaN,Jacob and Kiki Hantla,I love the Oribe bright blonde radiance spray....,No more brass!,1499472000,NaN,NaN
34277,5.0,True,"07 9, 2018",AAF5D1LTFGB7L,B01HGSJPMW,NaN,Libby Johnson,I love all of the Elemis products.,Five Stars,1531094400,NaN,NaN


In [14]:
#Number of records
print(len(df.index))

34278


In [15]:
#No. of reviews/counts per rating
df.groupby(by="overall").size()

overall
1.0     1095
2.0     1496
3.0     3884
4.0     7833
5.0    19970
dtype: int64

In [16]:
df.groupby(by="reviewText").size()

reviewText
!!!! Yes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              2
" wonder how you ever lived without it" you will be wondering a lot, 

In [17]:
#calculate average length (mean) of reviewText
df['reviewText'].str.len().mean()

489.144024514811

In [18]:
#df1[['reviewTime', 'year']]=df1['reviewTime'].str.split(',', 1, expand=True) 
#df1

#df[['team', 'conference']] = df['team'].str.split(',', 1, expand=True)

In [19]:
#Create a representation of our vocabulary-list the words present in the reviewtext
#if a word exists in reviewtext then 1 else 0
#Create a function where we pass entire reviewtext and handed back a vector representation of the review

In [20]:
#The first step of preparing text for modeling is tokenizing documents. In this step, words are isolated by 
#transforming a text string with many words, like a sentence or a document, into a list of individual words.
#When tokenizing:

#'A machine will never replace a teacher.'

#will be replaced with

#['A', 'machine', 'will', 'never', 'replace', 'a', 'teacher.']

In [21]:
#Load a large language model and assign it to the spacy language variable 'nlp_lg'
nlp_lg=spacy.load('en_core_web_lg')

In [22]:
#Define example sentence
text="I would like to have a dog and a cat in my home"

In [23]:
#Feed example sentence to the language model under 'nlp_lg'
#store the Spacy Doc object under the variable doc
doc=nlp_lg(text)

In [28]:
#show output
#The model has learned by observing the word in its context of occurences
doc 

I would like to have a dog and a cat in my home

In [29]:
#Retreive 6th token in the Doc object
doc[6]


dog

In [30]:
#Retreive 8th token in the Doc object
doc[9]

cat

In [31]:
#gives length of the vector
#This doc object is represented by a 300 dimensional attribute
doc.vector.shape

(300,)

In [32]:
#Retrieve the second Token in the doc and it's vector attributes 
#and retreive the first 30 dimensions of its vector representation
doc[1].vector[:30]

array([-0.049291,  0.20483 , -0.12808 , -0.086686, -0.11584 , -0.06313 ,
       -0.11188 , -0.15493 , -0.30914 ,  3.1112  , -0.062078,  0.080439,
        0.51842 ,  0.28753 , -0.33872 , -0.14584 , -0.23055 ,  0.73007 ,
       -0.38358 , -0.074632,  0.15303 ,  0.076039,  0.032505, -0.040018,
       -0.23066 ,  0.1503  , -0.24296 , -0.23203 ,  0.12932 , -0.1656  ],
      dtype=float32)

In [53]:
#words that are similar should be close to each other in the word embeddings
#similarity() method helps to take these words as input and calculate cosine similarity
#between their vector representations stored under the vector representation for Doc pbjects
#Calculate similarity between Dog and Cat which is .8 
#They are similar as both tokens are both household pets

dog=doc[6]
cat=doc[9]
dog.similarity(cat)

0.8016855120658875

In [34]:
dog.similarity(dog)

1.0

In [40]:
#Create a list of noun phrases from the doc
n_chunks=list(doc.noun_chunks)
n_chunks

[I, a dog, a cat, my home]

In [41]:
#compare similarity of the vector representations for the first and last noun
n_chunks[0].similarity(n_chunks[3])

0.496260404586792

In [42]:
#compare similarity of the vector representations for the first and last noun
n_chunks[1].similarity(n_chunks[2])

0.904877781867981

In [43]:
df_final=df[["overall","reviewText"]] 
df_final=df_final.dropna()

In [44]:
print(df_final)


       overall                                         reviewText
0          5.0  This handcream has a beautiful fragrance. It d...
1          5.0  wonderful hand lotion, for seriously dry skin,...
2          5.0  Best hand cream around.  Silky, thick, soaks i...
3          5.0                                           Thanks!!
4          5.0  Great hand lotion.  Soaks right in and leaves ...
...        ...                                                ...
34273      4.0  I like to use moisturizers and sunscreens that...
34274      4.0  I wouldn't be able to afford this if not asked...
34275      1.0                                        Did nothing
34276      5.0  I love the Oribe bright blonde radiance spray....
34277      5.0                 I love all of the Elemis products.

[34265 rows x 2 columns]


In [45]:
#Pull reviews into a panada dataframe
#feed reviews to model stacy
#get vector representation
#use Sklearn to train and test the model
#df_reviews=df_final["reviewText"]  
#df_reviews

In [46]:
#Create a vector of words
#Feed reviews column from Luxury Brand Pandas DF to the Stacy language model under 'nlp_lg'
#Use apply function which takes reviews column Luxury Brand DF; iterates thru the rows in DF in an efficient way;
#passing reviewText column

df_final.vector=df_final.reviewText.apply(nlp_lg)


/tmp/ipykernel_6094/3788551830.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_final.vector=df_final.reviewText.apply(nlp_lg)


In [47]:
df_final.vector

0        (This, handcream, has, a, beautiful, fragrance...
1        (wonderful, hand, lotion, ,, for, seriously, d...
2        (Best, hand, cream, around, .,  , Silky, ,, th...
3                                           (Thanks, !, !)
4        (Great, hand, lotion, .,  , Soaks, right, in, ...
                               ...                        
34273    (I, like, to, use, moisturizers, and, sunscree...
34274    (I, would, n't, be, able, to, afford, this, if...
34275                                       (Did, nothing)
34276    (I, love, the, Oribe, bright, blonde, radiance...
34277         (I, love, all, of, the, Elemis, products, .)
Name: reviewText, Length: 34265, dtype: object

In [48]:
#gives length of the vector
#represented by a 300 dimensional attribute
df_final.vector.shape

(34265,)

In [49]:
 #try to use df_final.vector column in train the model 
df_final.vector[0]

This handcream has a beautiful fragrance. It doesnt stay on or protect your hands through washing.  This size is quite small.

In [50]:
 #try to use df_final.vector column in train the model 
df_final.vector[1]

wonderful hand lotion, for seriously dry skin, stays on a long time, a little goes a long long way.. go easy.. wonderful scent.. maybe a bit strong at first, but dissipates after a while.

In [51]:
#Retrieve the first Token in the doc and it's vector attributes 
#and retreive the first 30 dimensions of its vector representation
df_final.vector[0].vector[:30]

array([-2.39773318e-02,  1.60133287e-01, -1.69866487e-01, -1.19961366e-01,
        3.71128134e-02, -8.83501619e-02,  2.16171713e-04, -2.29986593e-01,
       -3.56430374e-02,  1.86375070e+00, -1.39553532e-01,  8.89517367e-02,
       -3.03613804e-02, -8.78611356e-02, -9.06591788e-02, -8.26632604e-02,
       -9.61264819e-02,  1.28135312e+00, -5.18032312e-02,  2.41618827e-02,
       -5.21444231e-02,  2.68253181e-02, -6.32234442e-04,  9.43491831e-02,
        6.51731193e-02,  5.83816580e-02, -7.97323436e-02, -5.04485629e-02,
       -1.36352759e-02, -1.99832534e-03], dtype=float32)

In [52]:
#words that are similar should be close to each other in the embeddings
#similarity() method helps to take these words as input and calculate cosine similarity
#between their vector representations stored under the vector representation for Doc pbjects
#Calculate similarity between 2 reviews
#They are similar as both are about Handcream/lotion

r1=df_final.vector[0]
r2=df_final.vector[1]
r1.similarity(r2)

0.9326830525839549

In [217]:
#create set which only has keys
#vocab_set=set() 
#for review in df_final.vector:  
#    vocab_set.update(set(review.split()))

In [218]:
#len(vocab_set)

In [219]:
#vocab_set

In [220]:
#vector of the words
#make a dict of keys
#def create_vector_of_words(vocab,review): 
#    vector=dict.fromkeys(vocab,0)
#    for word in review:  
#       vector[word] = vector[word]+1
        
         
#    return vector
    
   

In [221]:
#I want to create a dict from a set of words
#want to find out if word exists
#v1=create_vector_of_words(vocab_set,df_reviews[0].split())

In [222]:
#bag of words representation on our data
#v1['handcream']

In [223]:
#print(df_reviews[0])

In [224]:
#dict.fromkeys(vocab_set,0)

In [242]:
#Use sklearn library ; uses C ; Initialize a vectorizer
vectorizer = CountVectorizer()

In [243]:
#create numpy array which uses this list of reviews
#reviews=np.array(df_reviews)
reviews=np.array(df_final.vector)


In [244]:
reviews

array([This handcream has a beautiful fragrance. It doesnt stay on or protect your hands through washing.  This size is quite small.,
       wonderful hand lotion, for seriously dry skin, stays on a long time, a little goes a long long way.. go easy.. wonderful scent.. maybe a bit strong at first, but dissipates after a while.,
       Best hand cream around.  Silky, thick, soaks in all the way leaving hands super soft.,
       ..., Did nothing,
       I love the Oribe bright blonde radiance spray. I've been using it twice a week for about 8 weeks and I didn't have brassiness between colors. I know some complained about dryness, but I didn't have that experience. I highly recommend this for use between colors.,
       I love all of the Elemis products.], dtype=object)

In [258]:
#pass a list of reviews and will create a BOW rep off our data
#pass set of vectors
bag_of_words=vectorizer.fit_transform(reviews)

AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'lower'

In [246]:
#sparse matrix-mostly 0 
#bag_of_words=bag_of_words.toarray()

In [247]:
#panada series of the reviews
df_ratings=df_final["overall"]  
df_ratings

0        5.0
1        5.0
2        5.0
3        5.0
4        5.0
        ... 
34273    4.0
34274    4.0
34275    1.0
34276    5.0
34277    5.0
Name: overall, Length: 34265, dtype: float64

In [248]:
#create numpy array which uses this list of ratings
ratings=np.array(df_ratings)

In [249]:
ratings

array([5., 5., 5., ..., 1., 5., 5.])

In [250]:
len(ratings)

34265

In [251]:
#Train and test our model ;x inputs y outcome
#Split arrays into random train and test subsets.
x_train, x_test, y_train, y_test = train_test_split(df_final.vector, ratings)

In [252]:
y_test

array([4., 5., 5., ..., 5., 3., 5.])

In [253]:
y_train

array([5., 5., 4., ..., 4., 5., 5.])

In [254]:
len(x_train), len(y_train)

(25698, 25698)

In [255]:
len(x_test), len(y_test)

(8567, 8567)

In [256]:
#create instance of a model ; Initialising the model ; logisticregression (watch video) ; use tgidf
lr = LogisticRegression(C=10.0, random_state=1, solver='lbfgs', multi_class='ovr')

In [257]:
#Learning/Training of model ; reviews & scores
lr.fit(x_train,y_train)

ValueError: setting an array element with a sequence.

In [52]:
y_predict=lr.predict(x_test)

In [53]:
#prediction array
y_predict

array([5., 5., 5., ..., 4., 2., 5.])

In [58]:
#actual data 
y_test

array([5., 5., 4., ..., 5., 2., 5.])

In [ ]:
#Take a look at the prediction and then compare it with actual ratings  

#71% accuracy of the prediction; 
#The prediction percent is better score that 58% cause the model has actually learned 

metrics.accuracy_score(y_test,y_predict)

In [55]:
df_final['overall'].value_counts()

5.0    19963
4.0     7831
3.0     3880
2.0     1496
1.0     1095
Name: overall, dtype: int64

In [56]:
len(df_final)

34265

In [57]:
19963/34265

0.5826061578870567